In [40]:
import pandas as pd
import os

# Chemin vers le dossier contenant les fichiers CSV
dossier_csv = 'f1db-csv'

# Liste pour stocker les DataFrames
df = {}

# Parcourir tous les fichiers du dossier
for fichier in os.listdir(dossier_csv):
    if fichier.endswith('.csv'):
        # Construire le chemin complet du fichier
        chemin_fichier = os.path.join(dossier_csv, fichier)
        
        # Lire le fichier CSV dans un DataFrame
        nom_df = fichier[:-4]  # Enlever l'extension '.csv'
        if nom_df.startswith('f1db-'):
            nom_df = nom_df[5:]  # Retirer 'f1db-' du début
        df[nom_df] = pd.read_csv(chemin_fichier)

print(f"Nombre de fichiers CSV importés : {len(df)}")
print("Noms des df créés :")
for nom in df.keys():
    print(f"- {nom}")


Nombre de fichiers CSV importés : 39
Noms des df créés :
- grands-prix
- races-pit-stops
- races-starting-grid-positions
- races-warming-up-results
- seasons-entrants-tyre-manufacturers
- races-sprint-race-results
- races-qualifying-2-results
- tyre-manufacturers
- races-free-practice-3-results
- seasons-constructor-standings
- races-pre-qualifying-results
- constructors-previous-next-constructors
- drivers
- constructors
- races-qualifying-1-results
- races-driver-standings
- races-driver-of-the-day-results
- seasons-driver-standings
- races-constructor-standings
- points_systems
- continents
- races-sprint-starting-grid-positions
- races-sprint-qualifying-results
- races-free-practice-2-results
- seasons
- seasons-entrants-drivers
- seasons-entrants-constructors
- countries
- entrants
- drivers-family-relationships
- seasons-entrants
- circuits
- races-fastest-laps
- engine-manufacturers
- races
- races-qualifying-results
- races-free-practice-4-results
- races-race-results
- races-f

In [51]:
# Récupérer les résultats de la course
courses = df['races'][['id', 'year', 'round', 'circuitId', 'grandPrixId', 'date']]
resultats_course = df['races-race-results'][['raceId', 'year', 'positionText', 'driverId', 'constructorId', 'points', 'fastestLap']].copy()

# Ajouter le round et le circuitId à resultats_course
resultats_course = resultats_course.merge(courses[['id', 'round', 'circuitId']], left_on='raceId', right_on='id', how='left')
resultats_course.drop('id', axis=1, inplace=True)

# Récupérer les systèmes de points
points_systems = df['points_systems']

# Calculer les points pour chaque système
for systeme in points_systems.columns[1:]:  # Ignorer la colonne 'position'
    resultats_course.loc[:, f'points_{systeme}'] = resultats_course['positionText'].apply(lambda x: points_systems.loc[points_systems['position'] == int(x), systeme].values[0] if x.isdigit() and int(x) <= 20 else 0)

# Calculer les points cumulés par pilote et par saison pour chaque barème
points_cumules = resultats_course.groupby(['year', 'driverId'])

for systeme in points_systems.columns[1:]:
    resultats_course[f'points_cumules_{systeme}'] = points_cumules[f'points_{systeme}'].transform('cumsum')

# Afficher les résultats
resultats_course


,raceId,year,positionText,driverId,constructorId,points,fastestLap,round,circuitId,points_formula_2010,...,points_formula_1950,points_motogp,points_indycar,points_cumules_formula_2010,points_cumules_formula_2003,points_cumules_formula_1991,points_cumules_formula_1961,points_cumules_formula_1950,points_cumules_motogp,points_cumules_indycar
0,1,1950,1,nino-farina,alfa-romeo,9.0,True,1,silverstone,25,...,8,25,50,25,10,10,9,8,25,50
1,1,1950,2,luigi-fagioli,alfa-romeo,6.0,False,1,silverstone,18,...,6,20,40,18,8,6,6,6,20,40
2,1,1950,3,reg-parnell,alfa-romeo,4.0,False,1,silverstone,15,...,4,16,35,15,6,4,4,4,16,35
3,1,1950,4,yves-giraud-cabantous,talbot-lago,3.0,False,1,silverstone,12,...,3,13,32,12,5,3,3,3,13,32
4,1,1950,5,louis-rosier,talbot-lago,2.0,False,1,silverstone,10,...,2,11,30,10,4,2,2,2,11,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26686,1119,2024,16,valtteri-bottas,kick-sauber,NaN,False,18,marina-bay,0,...,0,0,8,0,0,0,0,0,12,142
26687,1119,2024,17,pierre-gasly,alpine,NaN,False,18,marina-bay,0,...,0,0,6,8,0,0,0,0,51,216
26688,1119,2024,18,daniel-ricciardo,rb,NaN,True,18,marina-bay,0,...,0,0,4,7,1,0,0,0,54,232
26689,1119,2024,19,kevin-magnussen,haas,NaN,False,18,marina-bay,0,...,0,0,2,6,1,0,0,0,46,202


In [55]:
resultats_course[resultats_course['raceId'] == 1119][['driverId', 'points_cumules_formula_2010', 'points_cumules_formula_2003', 'points_cumules_formula_1991', 'points_cumules_formula_1961', 'points_cumules_formula_1950', 'points_cumules_motogp', 'points_cumules_indycar']].sort_values(by='points_cumules_formula_2010', ascending=False)


,driverId,points_cumules_formula_2010,points_cumules_formula_2003,points_cumules_formula_1991,points_cumules_formula_1961,points_cumules_formula_1950,points_cumules_motogp,points_cumules_indycar
26672,max-verstappen,305,125,105,98,89,321,688
26671,lando-norris,266,110,83,80,76,289,631
26675,charles-leclerc,229,94,68,66,64,251,573
26673,oscar-piastri,225,92,65,63,61,258,597
26677,carlos-sainz-jr,177,70,42,41,37,196,486
26676,lewis-hamilton,162,60,39,37,32,203,512
26674,george-russell,147,56,30,29,26,177,458
26680,sergio-perez,130,51,28,28,27,167,427
26678,fernando-alonso,61,19,5,5,2,121,376
26684,lance-stroll,24,7,1,1,0,69,260
